In [1]:
!pip install influxdb-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 1.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 744.6/744.6 kB 6.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.8/217.8 kB 6.7 MB/s eta 0:00:00


In [22]:
import influxdb_client
import pandas as pd
from influxdb_client.client.write_api import SYNCHRONOUS
import random

In [41]:
org='2684bd5f2421cb07'
bucket = "test2"


client = influxdb_client.InfluxDBClient(
    url='https://eu-central-1-1.aws.cloud2.influxdata.com',
    token='vSqR-lzePjp98WqlrlQa03f4fULu0MeumLVjId5_COSdbY2amYdNu0f891QPkS0Rt1wMvAPYAyh0FoV4aZX3Ig==',
    org=org)

#write_api = client.write_api(write_options=SYNCHRONOUS)
#p = influxdb_client.Point("my_measurement").tag("location", "Berlin").field("temperature", random.randint(0, 50))
#write_api.write(bucket=bucket, org=org, record=p)

In [49]:
query_api = client.query_api()

query = f'from(bucket:"{bucket}")\
|> range(start: -1d)\
|> filter(fn:(r) => r._measurement == "my_measurement")\
|> filter(fn:(r) => r.location == "Prague" or r.location == "Berlin")\
|> filter(fn:(r) => r._field == "temperature")'

result = query_api.query(org=org, query=query)

results = []
for table in result:
  for record in table.records:
    results.append((record.get_field(), record.get_value(), record.values.get("location")))

df = pd.DataFrame(results, columns=['field', 'value', 'location'])
df.head(10)

,field,value,location
0,temperature,25.3,Prague
1,temperature,25.3,Prague
2,temperature,25.3,Prague
3,temperature,16.0,Berlin
4,temperature,32.0,Berlin
5,temperature,41.0,Berlin
6,temperature,44.0,Berlin
7,temperature,41.0,Berlin
8,temperature,6.0,Berlin
9,temperature,0.0,Berlin
